In [18]:
# !pip install langchain
# !pip install -U langchain-community
# !pip install pypdf
# !pip install textract
# !pip install python-dotenv
# !pip install openai
# !pip install tiktoken
# !pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.2/14.6 MB 3.1 MB/s eta 0:00:05
   -- ------------------------------------- 0.8/14.6 MB 8.8 MB/s eta 0:00:02
   ---- ----------------------------------- 1.8/14.6 MB 12.5 MB/s eta 0:00:02
   ----- ---------------------------------- 2.0/14.6 MB 11.8 MB/s eta 0:00:02
   ------- -------------------------------- 2.6/14.6 MB 11.1 MB/s eta 0:00:02
   --------- ------------------------------ 3.6/14.6 MB 12.6 MB/s eta 0:00:01
   ------------- -------------------------- 5.1/14.6 MB 15.5 MB/s eta 0:00:01
   --------------- ------------------------ 5.7/14.6 MB 15.3 MB/s eta 0:00:01
   ----------------- ---------------------- 6.4/14.6 MB 15.0 MB/s eta 0:00:01
   ------------------- -------------------- 7.0/14.6 MB 14.8 MB/s eta 0:00:01
   -------------------- ------------------- 7.5/14.6 MB 14.4 MB/s eta 0:00

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

C:\Users\georg\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key from environment variables
api_key = os.getenv('OPENAI_API_KEY')

# print(f'API Key: {api_key}')

## 1. Loading PDFs  and chunking with LangChain

In [4]:
loader = PyPDFLoader("./attention_is_all_you_need.pdf")
pages = loader.load_and_split()
pages[0]

Document(metadata={'source': './attention_is_all_you_need.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, t

In [5]:
# !pip install PyMuPDF

In [6]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    with fitz.open(pdf_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return text
text = extract_text_from_pdf("./attention_is_all_you_need.pdf")

with open('attention_is_all_you_need.txt', 'w', encoding='utf-8') as f:
    f.write(text)
    
with open('attention_is_all_you_need.txt', 'r', encoding='utf-8') as f:
    text = f.read()

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

def count_tokens(text:str) -> int:
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 24,
    length_function = count_tokens)

chunks = text_splitter.create_documents([text])

In [7]:
chunks[0]

Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, disp

In [ ]:
token_counts = [count_tokens(chunk.page_content) for chunk in chunks]

df = pd.DataFrame({'Token Count': token_counts})

df.hist(bins=40,)

plt.show()

## 2. Embed text and store embeddings

In [9]:
# Get embedding model
embeddings = OpenAIEmbeddings(api_key=api_key)

# Create vector database
db = FAISS.from_documents(chunks, embeddings)

C:\Users\georg\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [1]:
query = "Who created transformers?"
docs = db.similarity_search(query)
docs[0]
len(docs[0])

NameError: name 'db' is not defined

In [ ]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type='stuff')
query = "Who created transformers?"
docs = db.similarity_search(query)
chain.run(input_documents = docs, question= query)

In [ ]:
from IPython.display import display
import ipywidgets as widgets

qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.1), db.as_retiver())

In [ ]:
chat_history = []

def onsubmit(_):
    query = input_box.value
    input_box.value = ""
    
    if query.lower() == 'exit':
        print("Thank you for using the sate of the union chatbot")
        return
    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))
    
    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b font color="blue">Chatbot:</font></b> {result["answer"]}'))    
    
print("Welcome to the Transformers chatbot! Type 'exit' to stop.")
input_box = widgets.Text(placeholder = 'Please enter your question:')
input_box.on_submit(onsubmit)

display(input_box)